In [1]:
## An one-mode example
import numpy as np
import matplotlib.cm as cm
import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation, PillowWriter
from mpl_toolkits.mplot3d import Axes3D

# Generate random samples
np.random.seed(0)
category1 = np.random.normal(loc=1, scale=1, size=50)
category2 = np.random.normal(loc=-1, scale=1, size=50)
# category2 = np.concatenate([np.random.normal(loc=-1, scale=1, size=50), np.random.normal(loc=-3, scale=1, size=50)])

# Prepare data for logistic regression
X = np.append(category1, category2).reshape(-1, 1)
X = np.hstack((X, np.ones((X.shape[0], 1))))  # Add bias term
y = np.append(np.ones(category1.shape[0]), np.zeros(category2.shape[0])) 
# We are using 0-1 label here due to the usage of binary cross-entropy loss

# Weight record
weights_history = []


# Logistic Regression Functions
def sigmoid(z):
    return 1 / (1 + np.exp(-z))

def compute_loss(w, X, y, lambda_reg = 0.0):
    z = X.dot(w)
    predictions = sigmoid(z)
    return -np.mean(y * np.log(predictions+eps) + (1 - y) * np.log(1 - predictions+eps)) + 1/2*lambda_reg*w.T.dot(w)

def logistic_regression(X, y, lr=0.01, epochs=100, lambda_reg = 0.0):
    weights_history = []
    w = -np.ones(X.shape[1])
    for epoch in range(epochs):
        z = np.dot(X, w)
        predictions = sigmoid(z)
        gradient = np.dot(X.T, (predictions - y)) / y.size + lambda_reg * w
        w -= lr * gradient
        weights_history.append(w.copy())
    return weights_history


# Animation function
def create_fig():
    # Set up the figure for animation with a 16:9 aspect ratio
    fig = plt.figure(figsize=(18, 6))
    ax1 = fig.add_subplot(131)
    ax2 = fig.add_subplot(132, projection='3d')
    ax3 = fig.add_subplot(133)
    
    # Probability Curve
    ax1.scatter(category1, np.ones(category1.shape[0]), s = 5, label='Category 1')
    ax1.scatter(category2, -np.ones(category2.shape[0]), s = 5, label='Category 2')
    ax1.set_xlabel('x')
    ax1.set_ylabel('Prediction')
    ax1.set_title('Probability Curve')
    ax1.grid(True)
    x_values = np.linspace(-6, 6, 300)
    line, = ax1.plot([], [], color='red', lw=2, label='p(y=1|x; w)')
    line2, = ax1.plot([], [], color='blue', lw=2, label='p(y=-1|x; w)')
    ax1.legend()
    
    # 3D Loss
    w1_range = np.linspace(-10, 10, 100)
    w2_range = np.linspace(-10, 10, 100)
    W1, W2 = np.meshgrid(w1_range, w2_range)
    losses = np.array([compute_loss(np.array([w1, w2]), X, y) for w1, w2 in zip(np.ravel(W1), np.ravel(W2))])
    losses = losses.reshape(W1.shape)
    ax2.plot_surface(W1, W2, losses, cmap=cm.viridis, edgecolor='none')
    ax2.set_xlabel('w1')
    ax2.set_ylabel('w2')
    ax2.set_zlabel('Loss')
    ax2.set_title('Loss Function Landscape')
    scat3d = ax2.scatter([], [], [], color='red', s=50)
    
    # 2D Contour and Optimization Traj
    ax3.contour(W1, W2, losses, levels=10, linewidths=0.5, colors='k')
    contour = ax3.contourf(W1, W2, losses, levels=50, cmap='RdBu_r')
    ax3.set_xlabel('w1')
    ax3.set_ylabel('w2')
    ax3.set_title('Loss Function')
    scat2d = ax3.scatter([], [], color='red', s=50)
    loss_text = ax3.text(0.5, -9, '', fontsize=12)
    return fig, [ax1, ax2, ax3], [x_values, line, line2, scat3d, scat2d, loss_text]

def animate(i):
    global weights_history, ax1, ax2, ax3

    weights = weights_history[i]
    w1, w2 = weights
    weights_history.append([w1, w2])

    # Update probability curve
    y_values = sigmoid(x_values * w1 + w2)
    line.set_data(x_values, y_values)
    line2.set_data(x_values, 1-y_values)


    # Update 3D loss function plot
    scat3d._offsets3d = ([w1], [w2], [compute_loss(weights, X, y)+1]) # add offset to make the dot visible
    
    # Update 2D contour plot
    loss_val = compute_loss(weights, X, y)
    scat2d.set_offsets([w1, w2])
    ax3.plot(*zip(*weights_history[:i+1]), color='red', markersize=1)
    loss_text.set_text(f'Iter: {i:d} Loss: {loss_val:.4f}')
    print(f'Iter {i:d}: w {weights}, loss {loss_val:.4f}')
    return line, scat3d, scat2d, loss_text


eps = 1e-15

# Initialize weights generator
weights_history = logistic_regression(X, y, lr=.5, epochs=100)

# Create animation
fig, [ax1, ax2, ax3], [x_values, line, line2, scat3d, scat2d, loss_text] = create_fig()
ani = FuncAnimation(fig, animate, frames=50, interval=100, blit=True)

# Save the animation
ani.save('logistic_regression_training.mp4', writer='ffmpeg', dpi=300, fps=5)

plt.close(fig)  # Close the figure after saving the animation

Iter 0: w [-0.55178498 -0.91364436], loss 1.1628
Iter 0: w [-0.55178498 -0.91364436], loss 1.1628
Iter 0: w [-0.55178498 -0.91364436], loss 1.1628
Iter 0: w [-0.55178498 -0.91364436], loss 1.1628
Iter 1: w [-0.16478594 -0.81641667], loss 0.8714
Iter 2: w [ 0.1485189 -0.719854 ], loss 0.6801
Iter 3: w [ 0.38793539 -0.63542287], loss 0.5666
Iter 4: w [ 0.56853115 -0.56592831], loss 0.5001
Iter 5: w [ 0.70871398 -0.50874911], loss 0.4587
Iter 6: w [ 0.82165737 -0.46081005], loss 0.4311
Iter 7: w [ 0.915578   -0.41983206], loss 0.4117
Iter 8: w [ 0.99562034 -0.38423734], loss 0.3973
Iter 9: w [ 1.06513626 -0.35292041], loss 0.3864
Iter 10: w [ 1.12640724 -0.32508377], loss 0.3778
Iter 11: w [ 1.18104861 -0.30013449], loss 0.3710
Iter 12: w [ 1.23024331 -0.2776196 ], loss 0.3654
Iter 13: w [ 1.27488277 -0.25718487], loss 0.3608
Iter 14: w [ 1.31565518 -0.23854764], loss 0.3569
Iter 15: w [ 1.35310301 -0.22147849], loss 0.3537
Iter 16: w [ 1.38766146 -0.20578848], loss 0.3509
Iter 17: w [ 1.

In [2]:
## What if we use a very large learning rate?

# Initialize weights generator
weights_history = logistic_regression(X, y, lr=15, epochs=100)

# Create animation
fig, [ax1, ax2, ax3], [x_values, line, line2, scat3d, scat2d, loss_text] = create_fig()
ani = FuncAnimation(fig, animate, frames=50, interval=100, blit=True)

# Save the animation
ani.save('logistic_regression_training_large_l1.mp4', writer='ffmpeg', dpi=300, fps=5)

plt.close(fig)  # Close the figure after saving the animation

Iter 0: w [12.44645071  1.59066931], loss 1.0010
Iter 0: w [12.44645071  1.59066931], loss 1.0010
Iter 0: w [12.44645071  1.59066931], loss 1.0010
Iter 0: w [12.44645071  1.59066931], loss 1.0010
Iter 1: w [11.32183566  1.40742464], loss 0.9148
Iter 2: w [10.20313485  1.24005495], loss 0.8298
Iter 3: w [9.09313014 1.08719153], loss 0.7467
Iter 4: w [7.99601019 0.94639169], loss 0.6658
Iter 5: w [6.91830728 0.81387726], loss 0.5882
Iter 6: w [5.87054266 0.68448487], loss 0.5154
Iter 7: w [4.87007146 0.55242591], loss 0.4498
Iter 8: w [3.94582688 0.41410261], loss 0.3950
Iter 9: w [3.14531653 0.27433727], loss 0.3557
Iter 10: w [2.53906413 0.15341158], loss 0.3352
Iter 11: w [2.19433897 0.07977071], loss 0.3295
Iter 12: w [2.08217823 0.05574279], loss 0.3290
Iter 13: w [2.06581388 0.05209579], loss 0.3290
Iter 14: w [2.06441739 0.05184466], loss 0.3290
Iter 15: w [2.06431883 0.05179369], loss 0.3290
Iter 16: w [2.06430753 0.05180794], loss 0.3290
Iter 17: w [2.06430891 0.05179879], loss 

In [3]:
## A two-mode example
import numpy as np
import matplotlib.cm as cm
import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation, PillowWriter
from mpl_toolkits.mplot3d import Axes3D

# Generate random samples
np.random.seed(0)
category1 = np.concatenate([np.random.normal(loc=1, scale=.25, size=50), np.random.normal(loc=5, scale=.25, size=50)])
category2 = np.random.normal(loc=-1, scale=.5, size=50)

# Prepare data for logistic regression
X = np.append(category1, category2).reshape(-1, 1)
X = np.hstack((X, np.ones((X.shape[0], 1))))  # Add bias term
y = np.append(np.ones(category1.shape[0]), np.zeros(category2.shape[0])) 
# We are using 0-1 label here due to the usage of binary cross-entropy loss

# Weight record
weights_history = []


# Logistic Regression Functions
def sigmoid(z):
    return 1 / (1 + np.exp(-z))

def compute_loss(w, X, y, lambda_reg = 0.0):
    z = X.dot(w)
    predictions = sigmoid(z)
    return -np.mean(y * np.log(predictions+eps) + (1 - y) * np.log(1 - predictions+eps)) + 1/2*lambda_reg*w.T.dot(w)

def logistic_regression(X, y, lr=0.01, epochs=100, lambda_reg = 0.0):
    weights_history = []
    w = -np.ones(X.shape[1])
    for epoch in range(epochs):
        z = np.dot(X, w)
        predictions = sigmoid(z)
        gradient = np.dot(X.T, (predictions - y)) / y.size + lambda_reg * w
        w -= lr * gradient
        weights_history.append(w.copy())
    return weights_history


# Animation function
def create_fig():
    # Set up the figure for animation with a 18:6 aspect ratio
    fig = plt.figure(figsize=(18, 6))
    ax1 = fig.add_subplot(131)
    ax2 = fig.add_subplot(132, projection='3d')
    ax3 = fig.add_subplot(133)
    
    # Probability Curve
    ax1.scatter(category1, np.ones(category1.shape[0]), s = 5, label='Category 1')
    ax1.scatter(category2, -np.ones(category2.shape[0]), s = 5, label='Category 2')
    ax1.set_xlabel('x')
    ax1.set_ylabel('Prediction')
    ax1.set_title('Probability Curve')
    ax1.grid(True)
    x_values = np.linspace(-6, 6, 300)
    line, = ax1.plot([], [], color='red', lw=2, label='p(y=1|x; w)')
    line2, = ax1.plot([], [], color='blue', lw=2, label='p(y=-1|x; w)')
    ax1.legend()
    
    
    # 3D Loss
    w1_range = np.linspace(-10, 10, 100)
    w2_range = np.linspace(-10, 10, 100)
    W1, W2 = np.meshgrid(w1_range, w2_range)
    losses = np.array([compute_loss(np.array([w1, w2]), X, y) for w1, w2 in zip(np.ravel(W1), np.ravel(W2))])
    losses = losses.reshape(W1.shape)
    ax2.plot_surface(W1, W2, losses, cmap=cm.viridis, edgecolor='none')
    ax2.set_xlabel('w1')
    ax2.set_ylabel('w2')
    ax2.set_zlabel('Loss')
    ax2.set_title('Loss Function Landscape')
    scat3d = ax2.scatter([], [], [], color='red', s=50)
    
    # 2D Contour and Optimization Traj
    ax3.contour(W1, W2, losses, levels=10, linewidths=0.5, colors='k')
    contour = ax3.contourf(W1, W2, losses, levels=50, cmap='RdBu_r')
    ax3.set_xlabel('w1')
    ax3.set_ylabel('w2')
    ax3.set_title('Loss Function')
    scat2d = ax3.scatter([], [], color='red', s=50)
    loss_text = ax3.text(0.5, -9, '', fontsize=12)
    return fig, [ax1, ax2, ax3], [x_values, line, line2, scat3d, scat2d, loss_text]

def animate(i):
    global weights, ax1, ax2, ax3

    weights = weights_history[i]
    w1, w2 = weights
    weights_history.append([w1, w2])

    # Update probability curve
    y_values = sigmoid(x_values * w1 + w2)
    line.set_data(x_values, y_values)
    line2.set_data(x_values, 1-y_values)


    # Update 3D loss function plot
    scat3d._offsets3d = ([w1], [w2], [compute_loss(weights, X, y)+1]) # add offset to make the dot visible
    
    # Update 2D contour plot
    loss_val = compute_loss(weights, X, y)
    scat2d.set_offsets([w1, w2])
    ax3.plot(*zip(*weights_history[:i+1]), color='red', markersize=1)
    loss_text.set_text(f'Iter: {i:d} Loss: {loss_val:.4f}')
    print(f'Iter {i:d}: w {weights}, loss {loss_val:.4f}')
    return line, scat3d, scat2d, loss_text


eps = 1e-15

# Initialize weights generator
weights_history = logistic_regression(X, y, lr=1, epochs=200)

# Create animation
fig, [ax1, ax2, ax3], [x_values, line, line2, scat3d, scat2d, loss_text] = create_fig()
ani = FuncAnimation(fig, animate, frames=100, interval=100, blit=True)

# Save the animation
ani.save('logistic_regression_training_two_mode.mp4', writer='ffmpeg', dpi=300, fps=5)

plt.close(fig)  # Close the figure after saving the animation

Iter 0: w [ 1.12537604 -0.53050328], loss 0.2226
Iter 0: w [ 1.12537604 -0.53050328], loss 0.2226
Iter 0: w [ 1.12537604 -0.53050328], loss 0.2226
Iter 0: w [ 1.12537604 -0.53050328], loss 0.2226
Iter 1: w [ 1.29145316 -0.47712333], loss 0.1949
Iter 2: w [ 1.43081901 -0.4377318 ], loss 0.1754
Iter 3: w [ 1.55248219 -0.40806117], loss 0.1606
Iter 4: w [ 1.66106324 -0.38555442], loss 0.1490
Iter 5: w [ 1.75941251 -0.3685026 ], loss 0.1395
Iter 6: w [ 1.84947354 -0.3556981 ], loss 0.1315
Iter 7: w [ 1.93265465 -0.34625709], loss 0.1248
Iter 8: w [ 2.01001872 -0.33951608], loss 0.1189
Iter 9: w [ 2.08239196 -0.3349669 ], loss 0.1138
Iter 10: w [ 2.15043143 -0.3322139 ], loss 0.1093
Iter 11: w [ 2.21466925 -0.33094461], loss 0.1053
Iter 12: w [ 2.27554307 -0.33090929], loss 0.1017
Iter 13: w [ 2.33341747 -0.33190614], loss 0.0984
Iter 14: w [ 2.38859962 -0.33377058], loss 0.0955
Iter 15: w [ 2.44135089 -0.33636721], loss 0.0927
Iter 16: w [ 2.4918956  -0.33958381], loss 0.0902
Iter 17: w [ 